# Exploratory Analysis of Kronos Model

This notebook is for experimenting with the Kronos model. We will:
1. Load the pre-trained Kronos model and tokenizer.
2. Fetch financial data using `yfinance`.
3. Use the model to make predictions on the fetched data.
4. Visualize the results.

In [2]:
# !pip install yfinance

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import sys
import yfinance as yf

# To import from parent directory
sys.path.append("../")

from model import Kronos, KronosTokenizer, KronosPredictor

### 1. Load Model and Tokenizer

In [5]:
# Load Model and Tokenizer
tokenizer = KronosTokenizer.from_pretrained("NeoQuasar/Kronos-Tokenizer-base")
model = Kronos.from_pretrained("NeoQuasar/Kronos-small")

# Instantiate Predictor
predictor = KronosPredictor(model, tokenizer, device="cuda:0", max_context=1000)

  2025-10-07T07:03:24.664633Z  WARN  Reqwest(reqwest::Error { kind: Request, url: "https://cas-server.xethub.hf.co/reconstruction/37f0525a4e4797e437229dfddc6f9623eac479961a3f0313a6e2dcd0e96c6fc5", source: hyper_util::client::legacy::Error(Connect, Custom { kind: Other, error: Custom { kind: InvalidData, error: InvalidMessage(InvalidContentType) } }) }). Retrying...
    at /home/runner/work/xet-core/xet-core/cas_client/src/http_client.rs:226

  2025-10-07T07:03:24.664684Z  WARN  Retry attempt #0. Sleeping 599.739365ms before the next attempt
    at /root/.cargo/registry/src/index.crates.io-1949cf8c6b5b557f/reqwest-retry-0.7.0/src/middleware.rs:171

  2025-10-07T07:03:32.364004Z  WARN  Reqwest(reqwest::Error { kind: Request, url: "https://cas-server.xethub.hf.co/reconstruction/37f0525a4e4797e437229dfddc6f9623eac479961a3f0313a6e2dcd0e96c6fc5", source: hyper_util::client::legacy::Error(Connect, Custom { kind: Other, error: Custom { kind: InvalidData, error: InvalidMessage(InvalidContentTyp

RuntimeError: No CUDA GPUs are available

### 2. Fetch Data using yfinance

In [ ]:
# Fetch daily data for a stock
ticker = "AAPL"
data = yf.download(ticker, start="2020-01-01", end="2023-01-01", interval="1d")

# Prepare data for the model
data = data.rename(columns={
    "Open": "open",
    "High": "high",
    "Low": "low",
    "Close": "close",
    "Volume": "volume"
})
data['amount'] = data['close'] * data['volume']
data['timestamps'] = data.index

print(data.head())

### 3. Make Prediction

In [ ]:
# Prediction setup
lookback = 365
pred_len = 90

x_df = data.iloc[-lookback-pred_len:-pred_len]
y_df_ground_truth = data.iloc[-pred_len:]

x_timestamp = x_df['timestamps']
y_timestamp = y_df_ground_truth['timestamps']

# Make Prediction
pred_df = predictor.predict(
    df=x_df[['open', 'high', 'low', 'close', 'volume', 'amount']],
    x_timestamp=x_timestamp,
    y_timestamp=y_timestamp,
    pred_len=pred_len,
    T=1.0,
    top_p=0.9,
    sample_count=1,
    verbose=True
)

print("Forecasted Data Head:")
print(pred_df.head())

### 4. Visualize the Results

In [ ]:
def plot_prediction(historical_df, ground_truth_df, prediction_df):
    plt.figure(figsize=(15, 7))
    
    plt.plot(historical_df.index, historical_df['close'], label='Historical Data', color='blue')
    plt.plot(ground_truth_df.index, ground_truth_df['close'], label='Ground Truth', color='green')
    plt.plot(prediction_df.index, prediction_df['close'], label='Prediction', color='red', linestyle='--')
    
    plt.title(f'{ticker} Stock Price Prediction')
    plt.xlabel('Date')
    plt.ylabel('Price (USD)')
    plt.legend()
    plt.grid(True)
    plt.show()

plot_prediction(x_df, y_df_ground_truth, pred_df)